# Linear regression in PyTorch
Here the linear regression model is implemented using PyTorch

In [1]:
import numpy as np
import torch

In the example the objective is to predict the yields for oranges and apples given the values of temperature, rainfall and humidity. Therefore the (linear) models are going to be
$$yield_{apple}  = w_{11} * \text{temp} + w_{12} * \text{rainfall} + w_{13} * \text{humidity} + b_{1}$$
$$yield_{apple}  = w_{21} * \text{temp} + w_{22} * \text{rainfall} + w_{23} * \text{humidity} + b_{2}$$

## Input data
The input data are the following

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array(
    [[73, 67, 43],
    [91, 88, 64],
    [87, 134, 58],
    [102, 43, 37],
    [69, 96, 70]], 
    dtype='float32'
)

inputs

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

The targets (labels) are the following

In [3]:
# Targets (apples, oranges)
targets = np.array(
    [[56, 70],
    [81, 101],
    [119, 133],
    [22, 37],
    [103, 119]], 
    dtype='float32'
)

targets

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)

Transform the input and targets in tensors

In [4]:
inputs = torch.from_numpy(inputs)
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [5]:
targets = torch.from_numpy(targets)
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Weights and biases initialization
Weights and biases are initialized with random values following a normal distribution with mean 0 and standard deviation 1. 
- Weights are collected in a tensor having the where the first dimension (rows) corresponds to the number of models (2, i.e., the number of outputs) and the second dimension (columns) corresponds to the number of features (3, i.e., the number of inputs)
- Biases are collected in a tensor having the first dimension equal to the number of models (2)

In [6]:
w = torch.randn((2,3), requires_grad=True)
w

tensor([[ 1.0936,  1.3622,  1.1569],
        [ 0.7902, -1.3567,  0.4537]], requires_grad=True)

In [7]:
b = torch.randn((2), requires_grad=True)
b

tensor([-0.9619,  0.2343], requires_grad=True)

## The model

The model is 
$$X \cdot W^{T} + B$$
This model can be written in python as a function of <code>x</code>

In [8]:
def regression_model(x):
    return x @ w.T + b

where 
- <code>@</code> is the matrix multiplication operator
- <code>.T</code> (or <code>.t()</code>) outputs the transpose tensor

Moreover, notice that <code>b</code> is a vector, while the result of <code>x @ w.T</code> is a matrix: the summation is computed only after <code>b</code> is broadcasted on the rows to match the matrix shape

## The predictions
The predictions of the model are computed calling the function

In [9]:
preds = regression_model(inputs)
preds

tensor([[219.8848, -13.4744],
        [292.4705, -18.2152],
        [343.8163, -86.5086],
        [211.9650,  39.2817],
        [286.2501, -43.7318]], grad_fn=<AddBackward0>)

If the predictions are compared with the targets it can be seen that they are quite different. That's because the model has not been trained yet, and weights and biases still have random values

## Definition of a Loss function

Improving the model, it must be evaluated how well the model is performing. The model predictions are compared with the actual targets using the following method:

- Calculate the difference between the two matrices (`preds` and `targets`).
- Square all elements of the difference matrix to remove negative values.
- Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).
$$\frac{\sum{(\text{pred}-\text{target})^{2}}}{N}$$

In [10]:
def mse(t1, t2):
    diff = t1 - t2
    quad_diff = diff * diff
    mean_quad_diff = torch.sum(quad_diff) / diff.numel()
    return mean_quad_diff

So the loss, in this case, is

In [11]:
mse_loss = mse(preds, targets)

## Gradient computation

Since <code>w</code> and <code>b</code> have been defined with <code>requires_grad</code> equal to True, the gradient of the loss function is going to be computed w.r.t <code>w</code> and <code>b</code>

In [12]:
mse_loss.backward()

The gradients are stored in the <code>.grad</code> attribute of <code>w</code> and <code>b</code>

In [13]:
w.grad, b.grad

(tensor([[ 16557.4219,  17095.1172,  10695.3438],
         [ -9407.0449, -12204.4023,  -7051.4937]]),
 tensor([ 194.6774, -116.5297]))

### Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of weights and biases, and the objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

- **increasing** the element weight value slightly will **increase** the loss
- **decreasing** the element weight value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

- **increasing** the element weight value slightly will **decrease** the loss
- **decreasing** the element weight value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

Since PyTorch accumulates gradients, to procede the gradients of <code>w</code> and <code>b</code> have to be reset to 0

In [14]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


Then, we register the current loss to compare it later

In [15]:
old_mse_loss = mse(preds, targets)

## Step-by-step: train the model using gradient descent
Follow these steps to train the model
1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

### 1. Generate predictions

In [16]:
# Generate predictions
preds = regression_model(inputs)
preds

tensor([[219.8848, -13.4744],
        [292.4705, -18.2152],
        [343.8163, -86.5086],
        [211.9650,  39.2817],
        [286.2501, -43.7318]], grad_fn=<AddBackward0>)

### 2. Calculate the loss (and record the loss for comparison)

In [17]:
# Calculate the loss
mse_loss = mse(preds, targets)
print(mse_loss)
old_mse_loss = mse_loss

tensor(28763.8750, grad_fn=<DivBackward0>)


### 3. Compute gradients w.r.t the weights and biases

In [18]:
# Compute gradients
mse_loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 16557.4219,  17095.1172,  10695.3438],
        [ -9407.0449, -12204.4023,  -7051.4937]])
tensor([ 194.6774, -116.5297])


### 4 and 5. Adjust the weights by subtracting a small quantity proportional to the gradient and reset the gradients to zero

In [19]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

So the weights and biases are updated subtracting from them a value equal to the product between each own gradient and a very small number (`10^-5` in this case), to ensure that they don't get modified by a very large amount. In this way, a small step is taken in the downhill direction of the gradient, not a giant leap. This number is called the **learning rate** of the algorithm. 

`torch.no_grad` is a context manager used to indicate to PyTorch that it shouldn't track, calculate, or modify gradients while updating the weights and biases. In this mode, the result of every computation will have `requires_grad=False`, even when the inputs have `requires_grad=True`.

See https://pytorch.org/docs/stable/generated/torch.no_grad.html

### Compare the loss between the predictions taken before and after the weights and biases update

In [20]:
new_preds = regression_model(inputs)
new_mse_loss = mse(new_preds, targets)
print(f'old loss: {old_mse_loss}')
print(f'new loss: {new_mse_loss}')

old loss: 28763.875
new loss: 19948.755859375


The loss reduced by a lot

## Train for multiple epochs

To reduce the loss further, repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an **epoch**. 

Train the model for 1000 epochs.

In [42]:
# reset w and b
from tqdm.notebook import tqdm


w = torch.randn((2,3), requires_grad=True)
b = torch.randn((2), requires_grad=True)

def regression_model(inputs,w,b):
    return inputs@w.T+b

def mse(t1,t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

loss_history = []

for i in tqdm(range(1000)):

    # generate predictions
    preds = regression_model(inputs,w,b)
    
    # calculate the loss
    mse_loss = mse(preds,targets)
    loss_history.append(mse_loss)

    # compute the gradients
    mse_loss.backward()

    # update w and b
    with torch.no_grad():
        w -= w.grad*1e-5
        b -= b.grad*1e-5
        w.grad.zero_()
        b.grad.zero_()

print(f'starting loss\t{loss_history[0]}')
print(f'ending loss\t{loss_history[-1]}')

  0%|          | 0/1000 [00:00<?, ?it/s]

starting loss	34829.1484375
ending loss	7.860625267028809
